<div align="right">
  <img src="./imgs/EII-ULPGC-logo.jpeg" width="600">
</div>

# **Aprendizaje por refuerzo 4**

Hasta ahora hemos visto los conceptos y algoritmos clásicos del aprendizaje por refuerzo, pero cuando los problemas se vuelven más complejos y el número de estados se vuelve inmanejable, es necesario recurrir a técnicas más avanzadas. Veremos cómo los algoritmos clásicos, como el Q-Learning, pueden ser mejorados con redes neuronales profundas para manejar problemas más complejos.

### **DeepMind**

DeepMind se fundó en Londres en 2010 por Demis Hassabis, Shane Legg y Mustafa Suleyman con la ambición de construir inteligencia artificial de propósito general. En enero de 2014 Google la adquirió por una cifra estimada en ~£400 millones, integrándola después en Alphabet. 

Su primer gran hito científico fue el DQN: un agente de aprendizaje por refuerzo profundo capaz de dominar decenas de juegos de Atari directamente a partir de píxeles, publicado en Nature en 2015. Poco después llegó AlphaGo, que en 2016 derrotó al campeón mundial Lee Sedol; la línea siguió con AlphaGo Zero (2017), que aprendía “desde cero” sin partidas humanas, y AlphaZero (2017), que generalizó el enfoque a ajedrez y shōgi. 

En biología, AlphaFold revolucionó la predicción de estructuras de proteínas y en 2024 la evolución AlphaFold 3 amplió el alcance a complejos biomoleculares completos (proteínas, ácidos nucleicos y ligandos) con una arquitectura de difusión. 

En abril de 2023, Google fusionó el equipo Brain con DeepMind para crear Google DeepMind, liderado por Hassabis, con el mandato de acelerar el desarrollo de IA segura y más general; desde entonces ha impulsado modelos como Gemini.

Google DeepMind's Deep Q-learning playing Atari

https://www.youtube.com/watch?v=V1eYniJ0Rnk

## **Deep Q-Learning (DQL)**

El algoritmo Deep Q-Learning (DQL) es una técnica de aprendizaje por refuerzo que combina la arquitectura de las redes neuronales profundas con el método de Q-learning. Esta combinación permite que el algoritmo sea eficaz en entornos con estados de alta dimensión que serían difíciles de manejar con técnicas más tradicionales. Deep Q-Learning fue popularizado por DeepMind en 2013, especialmente con su éxito al entrenar modelos que podían jugar a varios juegos de Atari directamente desde los píxeles de entrada.

<br>
<div align="center">
    <img src="imgs/deep_q_learning_pong.gif" width="500px">
</div>

https://www.youtube.com/watch?v=TmPfTpjtdgg



Partimos de la **ecuación de Bellman**, que define cómo actualizamos el valor $ Q $ en el algoritmo Q-Learning:

$$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left[ R_{t+1} + \gamma \max_a Q(S_{t+1}, a) - Q(S_t, A_t) \right]$$

En lugar de usar una tabla $ Q $, en **Deep Q-Learning** utilizamos una red neuronal $ \hat{Q}_{\theta} $, donde la entrada es el estado $ s $ y la salida son los valores $ Q $ para cada posible acción $ a $. 


<div align="center">
    <img src="./imgs/deep.jpg" width="60%">
</div>

En el caso del ejemplo basta con una salida binaria, ya que solo hay dos posibles acciones: derecha o izquierda.

<br>
<div align="center">
    <img src="imgs/pong_deepQL.svg" width="700px">
</div>

Sin embargo, para que el entrenamiento sea estable y efectivo, necesitamos un **objetivo (target)** con el cual comparar nuestras predicciones. Para ello, utilizamos una **red neuronal adicional**, conocida como **red objetivo** $ Q_{\theta^-} $, que genera los valores Q objetivos. Esta red objetivo se actualiza periódicamente con los parámetros de la red principal.


En el **Q-Learning tradicional**, la tabla Q es una matriz que almacena los valores Q para cada par de estado-acción posible. Cada celda de esta tabla representa el valor esperado de realizar una acción específica en un estado dado, es decir, qué tan buena es esa acción en ese estado en términos de la recompensa futura esperada. Este enfoque funciona bien cuando el espacio de estados y acciones es pequeño, ya que se puede mantener una tabla explícita para todas las combinaciones posibles. Sin embargo, cuando el espacio de estados crece (como en problemas con muchas variables o en entornos continuos), la tabla Q se vuelve impráctica o imposible de manejar debido a su tamaño.

Aquí es donde entra en juego el **Deep Q-Learning**. En lugar de utilizar una tabla para almacenar los valores Q, se emplea una **red neuronal profunda** para aproximar la función Q. La red toma como entrada un estado y devuelve los valores Q para todas las acciones posibles. Esto tiene varias implicaciones clave:

1. **Generalización sobre el espacio de estados**: En lugar de almacenar valores para cada estado específico (como en la tabla Q), la red neuronal aprende patrones a partir de los estados, permitiendo que generalice a estados que no ha visto antes. Esto es especialmente útil cuando los estados no son discretos sino continuos o muy numerosos.
   
2. **Reducción del almacenamiento**: En vez de mantener una tabla que puede volverse extremadamente grande, la red neuronal compacta esta información en los pesos y conexiones entre las capas de la red. Esto hace que el enfoque sea más escalable para problemas complejos.

3. **Entrenamiento basado en experiencias**: La red neuronal se entrena a partir de ejemplos de pares de estado-acción y sus correspondientes valores Q (o más bien, sus aproximaciones), lo que le permite ajustar sus parámetros para aproximar mejor la función Q en todo el espacio de estados.

4. **Aproximación continua**: La red neuronal puede manejar de forma natural espacios de estado continuos, donde sería imposible definir una tabla Q, ya que no se pueden enumerar todos los estados posibles.


#### **Cálculo del Target**

El objetivo, o **target**, se calcula con la siguiente fórmula:

$$ y_i = r_i + \gamma \max_{a'} Q(s', a'; \theta^-) $$

Donde:
- $ r_i $ es la recompensa inmediata tras tomar la acción.
- $ \gamma $ es el factor de descuento que pondera las recompensas futuras.
- $ Q(s', a'; \theta^-) $ es el valor Q para el siguiente estado $ s' $, calculado por la red objetivo.

#### **Función de Pérdida (Loss)**

Para entrenar la red neuronal, minimizamos la siguiente función de pérdida:

$$ \mathcal{L(\theta)} = \mathbb{E} \left[ ( \hat{Q}(s, a; \theta_i) - y_i )^2 \right] $$

Esta pérdida mide la diferencia entre la predicción de la red principal y el valor objetivo calculado por la red objetivo.

---

### **Componentes de la Ecuación de Pérdida:**

1. **$ \hat{Q}(s, a; \theta) $**: Es el valor Q predicho por la red principal para el estado actual $ s $ y la acción $ a $, utilizando los parámetros actuales $ \theta $.
   
2. **$ Q(s', a'; \theta^-) $**: Es el valor Q calculado por la red objetivo para el siguiente estado $ s' $ y la mejor acción posible $ a' $, utilizando los parámetros $ \theta^- $.
   
3. **$ r $**: Es la recompensa inmediata recibida tras tomar la acción $ a $ en el estado $ s $.
   
4. **$ \gamma $**: Es el factor de descuento que controla cuánto valoramos las recompensas futuras.
   
5. **$ \mathcal{L}(\theta) $**: Es la función de pérdida que mide la discrepancia entre el valor predicho por la red principal y el valor objetivo.
   
6. **$ \mathbb{E} $**: Denota la expectativa matemática, lo que indica que estamos tomando el promedio de la pérdida sobre varias transiciones. Estas transiciones generalmente se seleccionan de un **minibatch** almacenado en el **replay buffer**.

---

### **Replay Buffer (Memoria de Repetición)**

El **replay buffer** es un componente clave en el entrenamiento de Deep Q-Learning. Es una memoria donde el agente almacena las transiciones $ (s, a, r, s') $ experimentadas durante el juego. Este enfoque permite romper la correlación entre transiciones consecutivas, ya que durante el entrenamiento, seleccionamos un minibatch aleatorio del buffer. Esto ayuda a estabilizar el proceso de entrenamiento y mejora la convergencia del modelo.

### **Actualización de la Red Objetivo**

Para evitar que la red se vuelva inestable debido a constantes actualizaciones, los parámetros de la **red objetivo** $ \theta^- $ se actualizan copiando los parámetros de la **red principal** $ \theta $ solo cada cierto número de pasos $ N $. Esto desacopla las predicciones del futuro (calculadas por la red objetivo) de las actualizaciones constantes de la red principal, proporcionando una referencia estable durante el entrenamiento.

### **Exploración vs. Explotación**

El agente utiliza una estrategia de **$ \epsilon $-greedy**, lo que significa que, en cada paso, tomará una acción aleatoria con probabilidad $ \epsilon $ (exploración) o seleccionará la acción con el mayor valor Q (explotación). Este parámetro $ \epsilon $ empieza siendo alto para fomentar la exploración, pero se va reduciendo con el tiempo para que el agente se enfoque en explotar las mejores acciones aprendidas.

La fórmula para el decaimiento de $ \epsilon $ es:

$$
\epsilon_t = \epsilon_{\text{final}} + (\epsilon_{\text{initial}} - \epsilon_{\text{final}}) \cdot e^{-t / \lambda}
$$

Donde:
- $ \epsilon_t $ es el valor de exploración en el tiempo $ t $.
- $ \lambda $ es la tasa de decaimiento.
  


## **Pong**

A continución se muestra un ejemplo de implementación de un agente DQL para jugar al Pong de Atari. Puedes decargar el código completo desde [este enlace](./staff/DQN.zip).

El siguiente vídeo muestra el agente en funcionamiento antes de que comience el entrenamiento:

<div align="center">
<video controls>
<source src="./imgs/principio_entrenamiento.mp4">
</video>
</div>

Y este vídeo muestra el agente después del entrenamiento:

<div align="center">
<video controls>
<source src="./imgs/final_entrenamiento.mp4">
</video>
</div>

https://karpathy.github.io/2016/05/31/rl/